## Математическая основа логистической регрессии

Логистическая регрессия — это **линейный классификатор**, который моделирует **вероятность принадлежности объекта к положительному классу** с помощью **логистической (сигмоидной) функции**.

### Основная идея

В отличие от линейной регрессии, логистическая регрессия **не предсказывает значение напрямую**, а оценивает **вероятность** P(y = 1 | x), где:  
- **x** = (x₁, x₂, ..., xₙ) — признаковый вектор (например, TF-IDF веса слов),  
- y ∈ {0, 1} — метка класса (0 — негативный, 1 — позитивный отзыв).

Модель вычисляет **линейную комбинацию признаков**:

$$
z = w_1 x_1 + w_2 x_2 + \ldots + w_n x_n + b
$$

Затем применяет **сигмоидную функцию** σ(z) = 1 / (1 + exp(-z)), чтобы преобразовать z в вероятность:

$$
P(y = 1 \mid \mathbf{x}) = \frac{1}{1 + e^{-(w_1 x_1 + \ldots + w_n x_n + b)}}
$$

Классификация происходит по порогу 0.5:  
- если вероятность ≥ 0.5 → **1 (позитивный)**,  
- иначе → **0 (негативный)**.

### Обучение: минимизация функции потерь

Модель обучается путём минимизации **логистических потерь** (binary cross-entropy):

$$
\mathcal{L} = -\frac{1}{m} \sum_{i=1}^{m} \left[ y^{(i)} \log(\hat{p}^{(i)}) + (1 - y^{(i)}) \log(1 - \hat{p}^{(i)}) \right]
$$

где:  
- m — число примеров,  
- y⁽ⁱ⁾ — истинная метка,  
- p̂⁽ⁱ⁾ — предсказанная вероятность.

Функция потерь **выпуклая**, поэтому оптимизация (градиентный спуск) находит **глобальный минимум**.

### Регуляризация

Для текстовых данных (тысячи признаков) часто используется **L2-регуляризация**:

$$
\mathcal{L}_{\text{reg}} = \mathcal{L} + \frac{\lambda}{2} (w_1^2 + w_2^2 + \ldots + w_n^2)
$$

где λ > 0 — параметр регуляризации (чем больше λ, тем сильнее штраф за большие веса).

### Почему подходит для анализа тональности?

- Отлично работает с **разреженными векторами** (TF-IDF),  
- Даёт **надёжные вероятности** (важно для ROC/AUC),  
- **Интерпретируема**: положительные веса → слова, связанные с позитивом, отрицательные → с негативом,  
- **Быстро обучается** даже на больших данных.

> **Интуиция**: каждое слово "голосует" за или против позитивной тональности, и вес показывает силу и направление этого голоса.

## Простой пример: как логистическая регрессия классифицирует отзыв?

Представим, что модель уже обучена и «знает», насколько каждое слово влияет на тональность.  
У неё есть **веса** (коэффициенты) для ключевых слов:

| Слово        | Вес (w) |
|--------------|--------|
| **amazing**  | +2.1   |
| **perfect**  | +1.8   |
| **boring**   | -1.9   |
| **bad**      | -2.0   |
| **movie**    | +0.1   |
| **not**      | -0.3   |

Теперь поступает новый отзыв:

> **Отзыв**: *"This movie is not amazing."*

### Как модель считает?

1. **Выделяет слова**: `["movie", "not", "amazing"]`
2. **Складывает веса**:  
   `z = вес("movie") + вес("not") + вес("amazing")`  
   `z = 0.1 + (-0.3) + 2.1 = 1.9`
3. **Пропускает через сигмоиду**:  
   `P(позитив) = 1 / (1 + exp(-1.9)) ≈ 0.87`

Вероятность позитивного отзыва — **87%**, поэтому модель предсказывает: **позитивный**.

> Но подожди! Отзыв на самом деле **ироничный** ("not amazing" = плохо).  
> Логистическая регрессия **не улавливает контекст и порядок слов**, если не использовать n-граммы!  
> Именно поэтому в нашем векторизаторе мы включили **би- и триграммы** — чтобы фраза *"not amazing"* получила **свой собственный вес** (скорее всего, отрицательный).

### Вывод

Модель работает как **взвешенный подсчёт слов**, где каждое слово «голосует» за позитив или негатив.  
С n-граммами она начинает «понимать» и устойчивые фразы — и становится гораздо точнее.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import re

# Загрузка данных

df = pd.read_csv('data/imdb_reviews.csv', encoding='utf-8')


def preprocessor(text):
    if isinstance(text, float):  # Защита от NaN
        return ""
    # Удаляем HTML-теги
    text = re.sub(r'<[^>]*>', '', text)
    
    # Ищем смайлики (эмотиконы) 
    emoticons = re.findall(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    
    # Удаляем все не-буквенные символы 
    text = re.sub(r'[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', '')
    
    return text.strip()


# Очищаем все отзывы
print("Очищаем текст...")
df['cleaned_review'] = df['review'].apply(preprocessor)


# Разделение на обучающую и тестовую выборки

X_train, X_test, y_train, y_test = train_test_split(
    df['cleaned_review'], df['sentiment'], test_size=0.2, random_state=42
)


# Векторизация текста

vectorizer = TfidfVectorizer(
    max_features=5000,      # берем 5000 самых частых слов
    ngram_range=(1, 3),     # учитываем 1,2,3-граммы
    stop_words='english'    # удаляем слова-паразиты
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


# Пример для Logistic Regression
param_grid = {'C': [0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=3, scoring='roc_auc')
grid.fit(X_train_vec, y_train)
print("Лучший C:", grid.best_params_)

FileNotFoundError: [Errno 2] No such file or directory: 'data/imdb_reviews.csv'